In [1]:
# IMPORTS
##########################

import agent
import environment
import doubledqn
import tools
import memory
import simulation
import multiprocessing
import pandas as pd
import csv
import os

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
from keras import optimizers 

def iter_params(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield instance

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# ONE RUN
#############################################

agent = simulation.simulator(
                     connection_label = "lonely_worker",
                     q_network_type = 'simple',
                     target_q_network_type = 'simple',
                     gamma = 0.99,
                     target_update_freq = 1000,
                     train_freq = 5,
                     num_burn_in = 300,
                     batch_size = 30,
                     optimizer = optimizers.RMSprop(lr=0.001, rho=0.95),
                     loss_func = "mse",
                     max_ep_length = 1000,
                     experiment_id = "Lonely Worker Test",
                     model_checkpoint = True,
                     opt_metric = None,
                     
                    # environment parameters
                     net_file = "cross.net.xml",
                     route_file = "cross.rou.xml",
                     demand = "nominal",
                     state_shape = (1,11),
                     num_actions = 2,
                     use_gui = False,
                     delta_time = 10,
                 
                    # memory parameters
                     max_size = 100000,
                     
                    # additional parameters
                 
                     policy = "epsGreedy",
                     eps = 0.5,
                     num_episodes = 20,
                     monitoring = True,
                     episode_recording = True)

agent.train()
# agent.load("./logs/First gridsearch/run_144/model_checkpoints/runFirst gridsearch_iter165000.h5")

# agent.ddqn.train(env = agent.env, num_episodes = 100, policy = agent.policy, connection_label = agent.connection_label)

evaluation_results = agent.evaluate(runs=2, use_gui= False)
evaluation_results
# data= agent.ddqn.evaluate(env = agent.env, policy = "greedy")

In [ ]:
# GRIDSEARCH
###############################################################

exp_id = "Testing_eps_greedy"

# to search tomorrow: optimiser, delta time, exploration decay, network architecture

param = {
    "batch_size" : [50,100], # 30,50,60,[70]
    "target_update_frequency" : [5000], # 10000,20000,30000,[40000]
    "gamma" : [0.99], # 0.98,0.99,0.995,[0.999]
    "train_freq" : [1,3], # 2,3,4,[5]
    "max_size" : [100000], # 20000,50000,70000,[100000]
    "max_ep_length" : [2000], # 1000,2000,3000,[4000]
    "policy" : ["epsGreedy"], # ["", epsgreedy_decay, "linDecEpsGreedy", 
    "eps" : [0.5, 0.1],
    "lr" : [0.0005]
}

param_grid = iter_params(**param)

def worker(input, output):
    """Runs through a chunk of the grid"""

    for position, args in iter(input.get, 'STOP'):
        result = worker_task(position, args)
        output.put(result)


def worker_task(position, args):
    """Tells the worker what to do with grid chunk"""
    # initialise all objects
    
    # print('Run', position + 1, '-- parameters', args)
    
    agent = simulation.simulator(
                    connection_label = position + 1,
                    q_network_type = 'simple',
                    target_q_network_type = 'simple',
                    gamma = args[2],
                    target_update_freq = args[1],
                    train_freq = args[3],
                    num_burn_in = 1000,
                    batch_size = args[0],
                    optimizer = optimizers.RMSprop(lr= args[8], rho=0.95),
                    loss_func = "mse",
                    max_ep_length = args[5],
                    experiment_id = exp_id,
                    model_checkpoint = True,
                    opt_metric = None,

                   # environment parameters
                    net_file = "cross.net.xml",
                    route_file = "cross.rou.xml",
                    demand = "nominal",
                    state_shape = (1,11),
                    num_actions = 2,
                    use_gui = False,
                    delta_time = 10,

                   # memory parameters
                    max_size = args[4],

                   # additional parameters

                    policy = args[6],
                    eps = args[7],
                    num_episodes = 400,
                    monitoring = True)
    
    # print("training agent", position + 1)
    agent.train()
    # print("evaluating agent", position + 1)
    
    evaluation_results = agent.evaluate(runs = 5)

    return (multiprocessing.current_process().name, position + 1, args, evaluation_results["unfinished_runs"], evaluation_results["average_delay"])


def gridsearch(param_grid):
    """Runs a parallelised gridsearch"""

    number_of_processes = multiprocessing.cpu_count()

    # Set up task list
    tasks = [(idx, val) for idx, val in enumerate(param_grid)]

    # Create queues
    task_queue = multiprocessing.Queue()
    done_queue = multiprocessing.Queue()

    # Submit tasks
    for task in tasks:
        task_queue.put(task)
    # Start worker processes
    for i in range(number_of_processes):
        print('Started process #', i + 1)
        multiprocessing.Process(target = worker,
                                args = (task_queue, done_queue)).start()
    with open(os.path.join("./logs",exp_id,"GS_results"), "a",newline='') as file:
        writer = csv.writer(file, dialect = 'excel')
        writer.writerow(["Process", "Run", "Parameters", "Result"])
    
    # Get and print results
    results = []
    for i in range(len(tasks)):
        results.append(done_queue.get())
        with open(os.path.join("./logs",exp_id,"GS_results"), "a",newline='') as file:
            writer = csv.writer(file, dialect = 'excel')
            writer.writerow(results[-1])                
        print('%s -- [RESULTS]: Run %s -- Parameters %s -- Mean duration %6.0f' % results[-1])
        
    # Tell child processes to stop
    for i in range(number_of_processes):
        task_queue.put('STOP')

    # Now combine the results
    scores = [result[-1] for result in results]
    lowest = min(scores)
    winner = results[scores.index(lowest)]
    return winner, results

multiprocessing.freeze_support()
winner, results = gridsearch(param_grid)
winner

Started process # 1
Started process # 2
Started process # 3
Started process # 4
Started process # 5
Started process # 6
Started process # 7
Started process # 8
Episode finished during memory replay fill. Starting new episode...
Episode finished during memory replay fill. Starting new episode...
Episode finished during memory replay fill. Starting new episode...
Episode finished during memory replay fill. Starting new episode...
Episode finished during memory replay fill. Starting new episode...
Episode finished during memory replay fill. Starting new episode...
Episode finished during memory replay fill. Starting new episode...
Episode finished during memory replay fill. Starting new episode...
Episode finished during memory replay fill. Starting new episode...
Episode finished during memory replay fill. Starting new episode...
Episode finished during memory replay fill. Starting new episode...
Episode finished during memory replay fill. Starting new episode...
Episode finished during 

Run 1 -- running episode 40 / 400
Run 3 -- running episode 49 / 400
Run 7 -- running episode 47 / 400
Run 7 -- running episode 48 / 400
Run 3 -- running episode 50 / 400
Run 5 -- running episode 32 / 400
Run 1 -- running episode 41 / 400
Run 8 -- running episode 13 / 400
Run 7 -- running episode 49 / 400
Run 2 -- running episode 11 / 400
Run 3 -- running episode 51 / 400
Run 1 -- running episode 42 / 400
Run 5 -- running episode 33 / 400
Run 7 -- running episode 50 / 400
Run 6 -- running episode 9 / 400
Run 3 -- running episode 52 / 400
Run 4 -- running episode 14 / 400
Run 7 -- running episode 51 / 400
Run 1 -- running episode 43 / 400
Run 3 -- running episode 53 / 400
Run 5 -- running episode 34 / 400
Run 7 -- running episode 52 / 400
Run 3 -- running episode 54 / 400
Run 1 -- running episode 44 / 400
Run 7 -- running episode 53 / 400
Run 5 -- running episode 35 / 400
Run 8 -- running episode 14 / 400
Run 3 -- running episode 55 / 400
Run 1 -- running episode 45 / 400
Run 7 -- runnin

Run 3 -- running episode 107 / 400
Run 8 -- running episode 26 / 400
Run 1 -- running episode 86 / 400
Run 7 -- running episode 102 / 400
Run 3 -- running episode 108 / 400
Run 7 -- running episode 103 / 400
Run 5 -- running episode 67 / 400
Run 1 -- running episode 87 / 400
Run 3 -- running episode 109 / 400
Run 1 -- running episode 88 / 400
Run 3 -- running episode 110 / 400
Run 7 -- running episode 104 / 400
Run 5 -- running episode 68 / 400
Run 4 -- running episode 28 / 400
Run 2 -- running episode 22 / 400
Run 1 -- running episode 89 / 400
Run 3 -- running episode 111 / 400
Run 7 -- running episode 105 / 400
Run 8 -- running episode 27 / 400
Run 3 -- running episode 112 / 400
Run 1 -- running episode 90 / 400
Run 5 -- running episode 69 / 400
Run 6 -- running episode 18 / 400
Run 7 -- running episode 106 / 400
Run 3 -- running episode 113 / 400
Run 1 -- running episode 91 / 400
Run 5 -- running episode 70 / 400
Run 3 -- running episode 114 / 400
Run 7 -- running episode 107 / 400


Run 1 -- running episode 140 / 400
Run 3 -- running episode 161 / 400
Run 5 -- running episode 102 / 400
Run 1 -- running episode 141 / 400
Run 3 -- running episode 162 / 400
Run 7 -- running episode 152 / 400
Run 1 -- running episode 142 / 400
Run 5 -- running episode 103 / 400
Run 7 -- running episode 153 / 400
Run 2 -- running episode 32 / 400
Run 3 -- running episode 163 / 400
Run 8 -- running episode 39 / 400
Run 4 -- running episode 41 / 400
Run 1 -- running episode 143 / 400
Run 5 -- running episode 104 / 400
Run 3 -- running episode 164 / 400
Run 1 -- running episode 144 / 400
Run 7 -- running episode 154 / 400
Run 6 -- running episode 26 / 400
Run 5 -- running episode 105 / 400
Run 3 -- running episode 165 / 400
Run 1 -- running episode 145 / 400
Run 7 -- running episode 155 / 400
Run 3 -- running episode 166 / 400
Run 5 -- running episode 106 / 400
Run 1 -- running episode 146 / 400
Run 7 -- running episode 156 / 400
Run 3 -- running episode 167 / 400
Run 1 -- running episode

Run 5 -- running episode 143 / 400
Run 1 -- running episode 194 / 400
Run 3 -- running episode 213 / 400
Run 4 -- running episode 53 / 400
Run 7 -- running episode 199 / 400
Run 1 -- running episode 195 / 400
Run 5 -- running episode 144 / 400
Run 3 -- running episode 214 / 400
Run 7 -- running episode 200 / 400
Run 1 -- running episode 196 / 400
Run 3 -- running episode 215 / 400
Run 5 -- running episode 145 / 400
Run 8 -- running episode 51 / 400
Run 7 -- running episode 201 / 400
Run 1 -- running episode 197 / 400
Run 3 -- running episode 216 / 400
Run 5 -- running episode 146 / 400
Run 7 -- running episode 202 / 400
Run 1 -- running episode 198 / 400
Run 3 -- running episode 217 / 400
Run 2 -- running episode 42 / 400
Run 4 -- running episode 54 / 400
Run 7 -- running episode 203 / 400
Run 5 -- running episode 147 / 400
Run 1 -- running episode 199 / 400
Run 3 -- running episode 218 / 400
Run 7 -- running episode 204 / 400
Run 1 -- running episode 200 / 400
Run 3 -- running episode

Run 3 -- running episode 265 / 400
Run 1 -- running episode 246 / 400
Run 7 -- running episode 250 / 400
Run 3 -- running episode 266 / 400
Run 1 -- running episode 247 / 400
Run 7 -- running episode 251 / 400
Run 5 -- running episode 183 / 400
Run 2 -- running episode 51 / 400
Run 4 -- running episode 66 / 400
Run 3 -- running episode 267 / 400
Run 1 -- running episode 248 / 400
Run 7 -- running episode 252 / 400
Run 1 -- running episode 249 / 400
Run 3 -- running episode 268 / 400
Run 5 -- running episode 184 / 400
Run 7 -- running episode 253 / 400
Run 6 -- running episode 41 / 400
Run 8 -- running episode 63 / 400
Run 1 -- running episode 250 / 400
Run 7 -- running episode 254 / 400
Run 3 -- running episode 269 / 400
Run 5 -- running episode 185 / 400
Run 1 -- running episode 251 / 400
Run 7 -- running episode 255 / 400
Run 3 -- running episode 270 / 400
Run 4 -- running episode 67 / 400
Run 7 -- running episode 256 / 400
Run 5 -- running episode 186 / 400
Run 1 -- running episode 

Run 7 -- running episode 302 / 400
Run 2 -- running episode 64 / 400
Run 1 -- running episode 300 / 400
Run 5 -- running episode 215 / 400
Run 3 -- running episode 317 / 400
Run 7 -- running episode 303 / 400
Run 2 -- running episode 65 / 400
Run 1 -- running episode 301 / 400
Run 2 -- running episode 66 / 400
Run 3 -- running episode 318 / 400
Run 5 -- running episode 216 / 400
Run 1 -- running episode 302 / 400
Run 7 -- running episode 304 / 400
Run 4 -- running episode 79 / 400
Run 7 -- running episode 305 / 400
Run 2 -- running episode 67 / 400
Run 3 -- running episode 319 / 400
Run 1 -- running episode 303 / 400
Run 5 -- running episode 217 / 400
Run 8 -- running episode 75 / 400
Run 7 -- running episode 306 / 400
Run 3 -- running episode 320 / 400
Run 2 -- running episode 68 / 400
Run 1 -- running episode 304 / 400
Run 2 -- running episode 69 / 400
Run 1 -- running episode 305 / 400
Run 3 -- running episode 321 / 400
Run 7 -- running episode 307 / 400
Run 5 -- running episode 218

Run 1 -- running episode 347 / 400
Run 6 -- running episode 55 / 400
Run 7 -- running episode 349 / 400
Run 5 -- running episode 243 / 400
Run 2 -- running episode 104 / 400
Run 3 -- running episode 364 / 400
Run 1 -- running episode 348 / 400
Run 7 -- running episode 350 / 400
Run 3 -- running episode 365 / 400
Run 2 -- running episode 105 / 400
Run 5 -- running episode 244 / 400
Run 1 -- running episode 349 / 400
Run 8 -- running episode 85 / 400
Run 7 -- running episode 351 / 400
Run 2 -- running episode 106 / 400
Run 3 -- running episode 366 / 400
Run 1 -- running episode 350 / 400
Run 4 -- running episode 90 / 400
Run 7 -- running episode 352 / 400
Run 2 -- running episode 107 / 400
Run 5 -- running episode 245 / 400
Run 3 -- running episode 367 / 400
Run 1 -- running episode 351 / 400
Run 7 -- running episode 353 / 400
Run 2 -- running episode 108 / 400
Run 1 -- running episode 352 / 400
Run 3 -- running episode 368 / 400
Run 5 -- running episode 246 / 400
Run 7 -- running episod

In [ ]:
omega_test = 1-1e-9
omega_test

In [ ]:
decay_test = []
eps_test = 0.8
for i in range(300000):
    eps_test *= omega_test ** i
    decay_test.append(eps_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(300000),decay_test)

In [ ]:
first_results = pd.DataFrame(results).sort_values(by = 3)